In [18]:
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from sklearn.datasets import load_iris
from sklearn import preprocessing

In [19]:
# Import dataset with panda
df = pd.read_csv('output.csv')
features_considered = ['embb_req', 'embb_giv', 'embb_true', 'urllc_req', 'urllc_giv', 'urllc_true','miot_req', 'miot_giv', 'miot_true','mmtc_req', 'mmtc_giv', 'mmtc_true','voice_req', 'voice_giv', 'voice_true', 'band_rem']
features = df[features_considered]
features.index = df['round']
features.head()


,embb_req,embb_giv,embb_true,urllc_req,urllc_giv,urllc_true,miot_req,miot_giv,miot_true,mmtc_req,mmtc_giv,mmtc_true,voice_req,voice_giv,voice_true,band_rem
round,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,20000000000
1,53060623374,9672147890,17948900612,441558397,441558397,441558397,307329303,307329303,307329303,290564279,290564279,290564279,656440852,107000000,462206556,9181400131
2,61791841182,11251729806,18648148231,127393646,127393646,127393646,118367187,118367187,118367187,125391451,125391451,125391451,667700752,128000000,440998732,8249117910
3,53746066805,11841087971,18603564358,187138532,187138532,187138532,172638841,172638841,172638841,156175157,156175157,156175157,590798086,136000000,425685025,7506959499
4,45351762943,9316317701,18881524142,104094661,103813790,104089937,89447929,89447929,89447929,98968917,98968917,98968917,472310051,139000000,467457132,10252451663


In [20]:
# Parametes 
TRAIN_SPLIT = 30000
tf.random.set_seed(13)
past_history = 1
future_target = 0
step = 1
BATCH_SIZE = 30 # combines consecutive elements of this dataset into batches.
BUFFER_SIZE = 30000 # >= len dataset for perfetct shuffling
EVALUATION_INTERVAL = 3000 # steps_per_epoch = TotalTrainingSamples / TrainingBatchSize
VALIDATION_STEPS = 400 # validation_steps = TotalvalidationSamples / ValidationBatchSize
EPOCHS = 10
bs_capacity = 20000000000


In [21]:
# Normalize
dataset = features.values
dataset = dataset / bs_capacity

In [22]:
def multivariate_data(dataset, target, start_index, end_index, history_size, target_size, step):
  data = []
  labels = []

  start_index = start_index + history_size
  if end_index is None:
    end_index = len(dataset) - target_size

  for i in range(start_index, end_index):
    indices = range(i-history_size, i, step)

    data.append(dataset[indices])
    labels.append(target[i+target_size])

  return np.array(data), np.array(labels)


In [23]:
# Create validation and training set
x_train, y_train = multivariate_data(dataset[1:, (0, 1, 3, 4, 6, 7, 9, 10, 12 ,13, 15)], dataset[1:, (2, 5, 8, 11, 14)], 0, TRAIN_SPLIT, past_history,future_target, step)
x_val, y_val = multivariate_data(dataset[1:, (0, 1, 3, 4, 6, 7, 9, 10, 12 ,13, 15)], dataset[1:, (2, 5, 8, 11, 14)], TRAIN_SPLIT, None, past_history, future_target, step)

train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

val_data = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_data = val_data.batch(BATCH_SIZE).repeat()


In [24]:
# Create RNN
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(256, input_shape=x_train.shape[-2:])) # 4*(len features input * len output + input^2)
model.add(tf.keras.layers.Dense(5))

# Compile RNN
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='mae')

# Train RNN
history = model.fit(train_data, epochs=EPOCHS, steps_per_epoch=EVALUATION_INTERVAL, validation_data=val_data, validation_steps=VALIDATION_STEPS)


Train for 3000 steps, validate for 400 steps
Epoch 1/10
3000/3000 [==============================] - 65s 22ms/step - loss: 0.0101 - val_loss: 0.0079
Epoch 2/10
3000/3000 [==============================] - 43s 14ms/step - loss: 0.0066 - val_loss: 0.0069
Epoch 3/10
3000/3000 [==============================] - 44s 15ms/step - loss: 0.0059 - val_loss: 0.0067
Epoch 4/10
3000/3000 [==============================] - 42s 14ms/step - loss: 0.0055 - val_loss: 0.0065
Epoch 5/10
3000/3000 [==============================] - 44s 15ms/step - loss: 0.0053 - val_loss: 0.0065
Epoch 6/10
3000/3000 [==============================] - 40s 13ms/step - loss: 0.0051 - val_loss: 0.0058
Epoch 7/10
3000/3000 [==============================] - 40s 13ms/step - loss: 0.0049 - val_loss: 0.0060
Epoch 8/10
3000/3000 [==============================] - 40s 13ms/step - loss: 0.0048 - val_loss: 0.0062
Epoch 9/10
3000/3000 [==============================] - 46s 15ms/step - loss: 0.0047 - val_loss: 0.0060
Epoch 10/10
3000/30

In [25]:
#Example prediction
prediction = model.predict(x_val, batch_size=BATCH_SIZE)[0]

print(prediction)
print(y_val[0])

# Prediction with new input value
test = [[[35881556553, 8318124338, 310731060, 310731060, 268358393, 268358393, 197834085, 197834085, 385305779, 64000000, 10840952124]]]
test = np.array(test)/bs_capacity

prediction = model.predict(test)
print(prediction[0])
print(y_train[0]) # because I took the input of the first values in x_train for test


[0.95432276 0.00346038 0.00358214 0.00180863 0.00714597]
[0.53959284 0.00542234 0.00239058 0.00438351 0.00627515]
[0.94440514 0.00826166 0.00765055 0.00610377 0.01160236]
[0.93240741 0.00636968 0.00591836 0.00626957 0.02204994]


In [26]:
# Save trained model
model.save("model.h5")
